In [2]:
%conda install pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /Users/jingzhao/anaconda3

  added / updated specs:
    - pandas


The following packages will be UPDATED:

  pandas                              2.0.3-py311hdb55bb0_0 --> 2.1.4-py311hdb55bb0_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [3]:
%conda install statsmodels

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
%conda install scikit-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [54]:
import pandas as pd
industry_portfolio_file = '/Users/jingzhao/Desktop/FE PW1/Data/17 Industry Portfolios.CSV'
financial_data_file = '/Users/jingzhao/Desktop/FE PW1/Data/Financial Data.CSV'

industry_portfolio_data = pd.read_csv(industry_portfolio_file)
financial_data = pd.read_csv(financial_data_file)

In [55]:
# HML: Regressor Selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Data Construction
merged_data = pd.merge(industry_portfolio_data, financial_data[['Date', 'HML']], on='Date')
missing_values = merged_data.isnull().sum()

# Preparing the data for the model
X = merged_data.drop(['Date', 'HML'], axis=1)
y = merged_data['HML']

# Random Forest Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Feature importances
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Top 5 most important portfolios
top_5_portfolios = feature_importances.head(5)
top_5_portfolios

,Feature,Importance
12,Trans,0.217498
8,Steel,0.099219
10,Machn,0.091429
15,Finan,0.064864
16,Other,0.062537


In [56]:
# HML: Linear Regression
from sklearn.linear_model import LinearRegression
selected_features = top_5_portfolios['Feature'].tolist()

# Preparing the data with selected features
X_selected = merged_data[selected_features]

# Splitting the data into training and testing sets for the selected features
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
    X_selected, y, test_size=0.2, random_state=42)

# Creating and fitting the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_selected, y_train_selected)

# Making predictions and evaluating the model
y_pred_selected = lr_model.predict(X_test_selected)
mse_selected = mean_squared_error(y_test_selected, y_pred_selected)
rmse_selected = np.sqrt(mse_selected)

# Output the model performance
mse_selected, rmse_selected

import statsmodels.api as sm

# Creating and fitting the OLS model
ols_model = sm.OLS(y_train_selected, X_train_selected).fit()

# Printing the summary of the OLS model
ols_summary = ols_model.summary()
ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    HML   R-squared (uncentered):                   0.487
Model:                            OLS   Adj. R-squared (uncentered):              0.484
Method:                 Least Squares   F-statistic:                              175.6
Date:                Sat, 06 Jan 2024   Prob (F-statistic):                   2.37e-131
Time:                        10:11:03   Log-Likelihood:                         -2195.3
No. Observations:                 930   AIC:                                      4401.
Df Residuals:                     925   BIC:                                      4425.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Trans          0.2803      0.026     10.963      0.000       0.230       0.331
Steel          0.1721      0.018      9.318      0.000       0.136       0.208
Machn         -0.2657      0.027     -9.777      0.000      -0.319      -0.212
Finan          0.3271      0.027     12.266      0.000       0.275       0.379
Other         -0.5043      0.039    -12.929      0.000      -0.581      -0.428
==============================================================================
Omnibus:                      100.771   Durbin-Watson:                   1.915
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              348.924
Skew:                           0.488   Prob(JB):                     1.71e-76
Kurtosis:                       5.838   Cond. No.                         7.64
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
# HML Rolling Regression
from datetime import timedelta
def rolling_regression(data, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')

    unique_years = data['Date'].dt.year.unique()
    
    results = []
    start_year = unique_years[0]

    # Perform rolling regression
    while start_year + train_years + test_years <= unique_years[-1]:
        # Define training and testing periods
        train_start = pd.Timestamp(year=start_year, month=1, day=1)
        train_end = train_start + timedelta(days=365 * train_years)
        test_end = train_end + timedelta(days=365 * test_years)

        # Subset the data for training and testing
        train_data = data[(data['Date'] >= train_start) & (data['Date'] < train_end)]
        test_data = data[(data['Date'] >= train_end) & (data['Date'] < test_end)]

        # Fit the model
        X_train = train_data[selected_features]
        y_train = train_data['HML']
        model = LinearRegression().fit(X_train, y_train)

        # Predict on test data
        X_test = test_data[selected_features]
        y_test = test_data['HML']
        y_pred = model.predict(X_test)

        # Collect coefficients and predictions
        coefficients = model.coef_
        results.append({
            'train_start': train_start,
            'train_end': train_end,
            'test_end': test_end,
            'coefficients': coefficients,
            'predicted_HML': y_pred
        })

        # Move to the next period
        start_year += test_years

    return pd.DataFrame(results)

# Rolling regression for each scenario
results_5_year = rolling_regression(merged_data, train_years=5, test_years=5)
results_10_year = rolling_regression(merged_data, train_years=10, test_years=5)
results_20_year = rolling_regression(merged_data, train_years=20, test_years=5)

results_5_year.head(), results_10_year.head(), results_20_year.head()
# Extracting betas and predicted HML into separate dataframes for each time scheme

def extract_betas_and_predictions(results):
    betas = []
    predicted_HML = []

    for index, row in results.iterrows():
        betas.append({
            'train_start': row['train_start'],
            'train_end': row['train_end'],
            'test_end': row['test_end'],
            'Trans_beta': row['coefficients'][0],
            'Steel_beta': row['coefficients'][1],
            'Machn_beta': row['coefficients'][2],
            'Finan_beta': row['coefficients'][3],
            'Other_beta': row['coefficients'][4]
        })
        predicted_HML.extend(row['predicted_HML'])

    betas_df = pd.DataFrame(betas)
    predicted_HML_df = pd.DataFrame({'predicted_HML': predicted_HML})

    return betas_df, predicted_HML_df

# Extracting for each time scheme
betas_5_year, predicted_HML_5_year = extract_betas_and_predictions(results_5_year)
betas_10_year, predicted_HML_10_year = extract_betas_and_predictions(results_10_year)
betas_20_year, predicted_HML_20_year = extract_betas_and_predictions(results_20_year)

In [58]:
# Residual
def calculate_residuals(data, predicted_HML, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()
    residuals = []

    start_year = unique_years[0]
    predicted_index = 0

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_HML = test_data['HML'].values

        # Calculate residuals
        predicted_HML_values = predicted_HML['predicted_HML'].iloc[predicted_index:predicted_index + len(actual_HML)]
        residual = actual_HML - predicted_HML_values
        residuals.extend(residual)

        # Update indices
        predicted_index += len(actual_HML)
        start_year += test_years

    return pd.DataFrame({'residuals': residuals})

# Calculate residuals for each rolling scheme
residuals_5_year = calculate_residuals(merged_data, predicted_HML_5_year, 5, 5)
residuals_10_year = calculate_residuals(merged_data, predicted_HML_10_year, 10, 5)
residuals_20_year = calculate_residuals(merged_data, predicted_HML_20_year, 20, 5)

residual_summary_5_year = residuals_5_year.describe()
residual_summary_10_year = residuals_10_year.describe()
residual_summary_20_year = residuals_20_year.describe()

residual_summary_5_year, residual_summary_10_year, residual_summary_20_year

(         residuals
 count  1080.000000
 mean     -0.031597
 std       2.475316
 min     -13.710342
 25%      -1.332233
 50%      -0.075307
 75%       1.360415
 max      10.027822,
          residuals
 count  1020.000000
 mean     -0.098707
 std       2.148893
 min     -13.030419
 25%      -1.326080
 50%      -0.103802
 75%       1.141364
 max       9.186277,
         residuals
 count  900.000000
 mean    -0.130205
 std      2.086146
 min     -8.466385
 25%     -1.372565
 50%     -0.163566
 75%      1.103427
 max      8.381515)

In [59]:
# SSE for each time scheme
sse_5_year = np.sum(residuals_5_year['residuals'] ** 2)
sse_10_year = np.sum(residuals_10_year['residuals'] ** 2)
sse_20_year = np.sum(residuals_20_year['residuals'] ** 2)

sse_5_year, sse_10_year, sse_20_year

(6612.31704619941, 4715.415162630034, 3927.7100470840132)

In [60]:
# out-of-sample R-squared
def calculate_out_of_sample_r_squared(data, predicted_HML, residuals, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()

    total_sum_squares = 0
    residual_sum_squares = 0
    predicted_index = 0

    start_year = unique_years[0]

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_HML = test_data['HML'].values

        # Calculate total sum of squares and residual sum of squares
        mean_actual_HML = np.mean(actual_HML)
        total_sum_squares += np.sum((actual_HML - mean_actual_HML) ** 2)
        residual_sum_squares += np.sum(residuals['residuals'].iloc[predicted_index:predicted_index + len(actual_HML)] ** 2)

        # Update indices
        predicted_index += len(actual_HML)
        start_year += test_years

    return 1 - (residual_sum_squares / total_sum_squares)

# Calculate out-of-sample R-squared for each rolling scheme
r_squared_5_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_5_year, residuals_5_year, 5, 5)
r_squared_10_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_10_year, residuals_10_year, 10, 5)
r_squared_20_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_20_year, residuals_20_year, 20, 5)

r_squared_5_year, r_squared_10_year, r_squared_20_year

(0.4973857240612507, 0.42530172773361963, 0.4024258908893674)

In [61]:
# MOM Get Regressors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

merged_data = pd.merge(industry_portfolio_data, financial_data[['Date', 'MOM']], on='Date')
missing_values = merged_data.isnull().sum()
X = merged_data.drop(['Date', 'MOM'], axis=1)
y = merged_data['MOM']

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and fitting the Random Forest model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Making predictions and evaluating the model
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Output the model performance and the missing values information
mse, rmse, missing_values
# Getting feature importances from the Random Forest model
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Top 5 most important portfolios
top_5_portfolios = feature_importances.head(5)
top_5_portfolios

,Feature,Importance
12,Trans,0.162974
9,FabPr,0.103529
10,Machn,0.087123
15,Finan,0.081107
8,Steel,0.080031


In [62]:
# MOM: Linear Regression
from sklearn.linear_model import LinearRegression

selected_features = top_5_portfolios['Feature'].tolist()
X_selected = merged_data[selected_features]

# Splitting data
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
    X_selected, y, test_size=0.2, random_state=42)

# Creating and fitting the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_selected, y_train_selected)

# Making predictions and evaluating the model
y_pred_selected = lr_model.predict(X_test_selected)
mse_selected = mean_squared_error(y_test_selected, y_pred_selected)
rmse_selected = np.sqrt(mse_selected)

# Output the model performance
mse_selected, rmse_selected

import statsmodels.api as sm

# Creating and fitting the OLS model
ols_model = sm.OLS(y_train_selected, X_train_selected).fit()

# Printing the summary of the OLS model
ols_summary = ols_model.summary()
ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    MOM   R-squared (uncentered):                   0.217
Model:                            OLS   Adj. R-squared (uncentered):              0.213
Method:                 Least Squares   F-statistic:                              51.23
Date:                Sat, 06 Jan 2024   Prob (F-statistic):                    6.09e-47
Time:                        10:13:15   Log-Likelihood:                         -2642.8
No. Observations:                 930   AIC:                                      5296.
Df Residuals:                     925   BIC:                                      5320.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Trans         -0.1659      0.042     -3.956      0.000      -0.248      -0.084
FabPr          0.1316      0.044      3.018      0.003       0.046       0.217
Machn          0.1856      0.039      4.728      0.000       0.109       0.263
Finan         -0.2328      0.041     -5.715      0.000      -0.313      -0.153
Steel         -0.1492      0.030     -4.902      0.000      -0.209      -0.089
==============================================================================
Omnibus:                      311.184   Durbin-Watson:                   1.775
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2831.555
Skew:                          -1.258   Prob(JB):                         0.00
Kurtosis:                      11.169   Cond. No.                         5.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
# MOM Rolling Regression
from datetime import timedelta

def rolling_regression(data, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()
    
    results = []
    start_year = unique_years[0]

    # Perform rolling regression
    while start_year + train_years + test_years <= unique_years[-1]:
        train_start = pd.Timestamp(year=start_year, month=1, day=1)
        train_end = train_start + timedelta(days=365 * train_years)
        test_end = train_end + timedelta(days=365 * test_years)

        train_data = data[(data['Date'] >= train_start) & (data['Date'] < train_end)]
        test_data = data[(data['Date'] >= train_end) & (data['Date'] < test_end)]

        # Fit the model
        X_train = train_data[selected_features]
        y_train = train_data['MOM']
        model = LinearRegression().fit(X_train, y_train)

        # Predict on test data
        X_test = test_data[selected_features]
        y_test = test_data['MOM']
        y_pred = model.predict(X_test)

        # Collect coefficients and predictions
        coefficients = model.coef_
        results.append({
            'train_start': train_start,
            'train_end': train_end,
            'test_end': test_end,
            'coefficients': coefficients,
            'predicted_MOM': y_pred
        })

        # Move to the next period
        start_year += test_years

    return pd.DataFrame(results)

# Rolling regression for each scenario
results_5_year = rolling_regression(merged_data, train_years=5, test_years=5)
results_10_year = rolling_regression(merged_data, train_years=10, test_years=5)
results_20_year = rolling_regression(merged_data, train_years=20, test_years=5)

results_5_year.head(), results_10_year.head(), results_20_year.head()

def extract_betas_and_predictions(results):
    betas = []
    predicted_MOM = []

    for index, row in results.iterrows():
        betas.append({
            'train_start': row['train_start'],
            'train_end': row['train_end'],
            'test_end': row['test_end'],
            'Trans_beta': row['coefficients'][0],
            'Steel_beta': row['coefficients'][1],
            'Machn_beta': row['coefficients'][2],
            'Finan_beta': row['coefficients'][3],
            'Other_beta': row['coefficients'][4]
        })
        predicted_MOM.extend(row['predicted_MOM'])

    betas_df = pd.DataFrame(betas)
    predicted_MOM_df = pd.DataFrame({'predicted_MOM': predicted_MOM})

    return betas_df, predicted_MOM_df

# Extracting for each time scheme
betas_5_year, predicted_MOM_5_year = extract_betas_and_predictions(results_5_year)
betas_10_year, predicted_MOM_10_year = extract_betas_and_predictions(results_10_year)
betas_20_year, predicted_MOM_20_year = extract_betas_and_predictions(results_20_year)

In [39]:
# Residual
def calculate_residuals(data, predicted_MOM, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()
    residuals = []

    start_year = unique_years[0]
    predicted_index = 0

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_MOM = test_data['MOM'].values

        # Calculate residuals
        predicted_MOM_values = predicted_MOM['predicted_MOM'].iloc[predicted_index:predicted_index + len(actual_MOM)]
        residual = actual_MOM - predicted_MOM_values
        residuals.extend(residual)

        # Update indices
        predicted_index += len(actual_MOM)
        start_year += test_years

    return pd.DataFrame({'residuals': residuals})

# Calculate residuals for each rolling scheme
residuals_5_year = calculate_residuals(merged_data, predicted_MOM_5_year, 5, 5)
residuals_10_year = calculate_residuals(merged_data, predicted_MOM_10_year, 10, 5)
residuals_20_year = calculate_residuals(merged_data, predicted_MOM_20_year, 20, 5)

residuals_5_year.head(), residuals_10_year.head(), residuals_20_year.head()


(   residuals
 0  -3.057034
 1   5.482401
 2   0.420145
 3   0.416680
 4  -2.716594,
    residuals
 0  -5.889969
 1   7.021825
 2   4.742564
 3 -10.277711
 4  -3.687156,
    residuals
 0  -8.271724
 1  -1.951298
 2   1.862592
 3   1.541931
 4   1.127191)

In [64]:
# Residual Summary
residual_summary_5_year = residuals_5_year.describe()
residual_summary_10_year = residuals_10_year.describe()
residual_summary_20_year = residuals_20_year.describe()

residual_summary_5_year, residual_summary_10_year, residual_summary_20_year

(         residuals
 count  1080.000000
 mean     -0.031597
 std       2.475316
 min     -13.710342
 25%      -1.332233
 50%      -0.075307
 75%       1.360415
 max      10.027822,
          residuals
 count  1020.000000
 mean     -0.098707
 std       2.148893
 min     -13.030419
 25%      -1.326080
 50%      -0.103802
 75%       1.141364
 max       9.186277,
         residuals
 count  900.000000
 mean    -0.130205
 std      2.086146
 min     -8.466385
 25%     -1.372565
 50%     -0.163566
 75%      1.103427
 max      8.381515)

In [65]:
# SSE for each time scheme
sse_5_year = np.sum(residuals_5_year['residuals'] ** 2)
sse_10_year = np.sum(residuals_10_year['residuals'] ** 2)
sse_20_year = np.sum(residuals_20_year['residuals'] ** 2)

sse_5_year, sse_10_year, sse_20_year

(6612.31704619941, 4715.415162630034, 3927.7100470840132)

In [66]:
# Out-of-sample R-squared
def calculate_out_of_sample_r_squared(data, predicted_MOM, residuals, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()

    total_sum_squares = 0
    residual_sum_squares = 0
    predicted_index = 0

    start_year = unique_years[0]

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_MOM = test_data['MOM'].values

        # Calculate total sum of squares and residual sum of squares
        mean_actual_MOM = np.mean(actual_MOM)
        total_sum_squares += np.sum((actual_MOM - mean_actual_MOM) ** 2)
        residual_sum_squares += np.sum(residuals['residuals'].iloc[predicted_index:predicted_index + len(actual_MOM)] ** 2)

        # Update indices
        predicted_index += len(actual_MOM)
        start_year += test_years

    return 1 - (residual_sum_squares / total_sum_squares)

# Calculate out-of-sample R-squared for each rolling scheme
r_squared_5_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_5_year, residuals_5_year, 5, 5)
r_squared_10_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_10_year, residuals_10_year, 10, 5)
r_squared_20_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_20_year, residuals_20_year, 20, 5)

r_squared_5_year, r_squared_10_year, r_squared_20_year

(0.7104265785457922, 0.7125108563357591, 0.7074549705163533)